# Anomaly Detection with the T-Digest

The [t-digest](https://github.com/tdunning/t-digest) is a compact data structure for summarizing observed cumulative probability distributions. Like many data sketching structures, it is incremental, scalable, and parallel (although the implementation in this notebook is not parallel).

This notebook explores using T-Digests as an anomaly detection structure. T-Digests sketch a Cumulative Distribution Function, and so they can be used to generate anomaly scores using a CDF as we did for Gaussian data in the previous notebook. However, T-Digests will accurately sketch distributions of any shape, which makes them a very flexible anomaly detection tool.

In [ ]:
import math
import numpy as np
import scipy
import scipy.stats
import pandas as pd
import altair as alt
alt.renderers.enable("notebook")

# T-Digest

In the following cell we will import a `TDigest` class that implements the `update` operation for inserting data elements into the sketch. This tutorial implementation does not provide the `merge` operation, which is useful for combining partial results in a distributed computing setting.

For those interested in how t-digests operate, the implementation is in the `tdigest.py` file of this workshop repository.

In [ ]:
from detail.tdigest import TDigest

## Sketching data with the t-digest

Data in the real world rarely obeys exact parameteric distributions, such as Gaussian, Exponential, or Gamma.
In the following cells, we will generate some data that is a mixture of two Gamma distributions having different shapes.
The resulting data has a somewhat irregular distribution, but is non-negative and has a long tail, similar to system-generated distributions like query latency data.

In [ ]:
a1 = 1
a2 = 7
w1 = 0.5
w2 = 0.5

# Sample from a mixture of two gamma distributions
def mixsamp():
    r = scipy.stats.uniform.rvs(size=1)[0]
    if (r <= w1):
        return scipy.stats.gamma.rvs(a1, size=1)[0]
    else:
        return scipy.stats.gamma.rvs(a2, size=1)[0]

# Sketch some data sampled from this distribution with a t-digest
sketch = TDigest(compression = 0.1)
for p in [mixsamp() for x in range(100000)]:
    sketch.update(p)

# Visualizing the CDF
In the following cell we plot the t-digest CDF along side the true CDF, to demonstrate that the t-digest has provided an accurate sketch of this somewhat irregular distribution.

In [ ]:
xvals = np.arange(sketch.cdfi(0), sketch.cdfi(1)).tolist()
df = pd.DataFrame()
df["x"] = xvals + xvals
df["cdf"] = [sketch.cdf(x) for x in xvals] + [(w1 * scipy.stats.gamma.cdf(x, a1)) + (w2 * scipy.stats.gamma.cdf(x, a2)) for x in xvals]
df["src"] = (["tdigest"] * len(xvals)) + (["cdf"] * len(xvals))
alt.Chart(df).mark_line().encode(x="x", y="cdf", color="src")

# A T-Digest Anomaly Score
The next cell defines an anomaly detector class that implements a negative-log based anomaly score similar to the `anomaly2` method in our earlier Gaussian anomaly detector.
This detector uses a t-digest sketch to obtain its CDF values.
Another difference is that it assumes a single-tailed distribution, such that only large positive values are considered anomalous.

In [ ]:
class TDigestAnomalyDetector(object):
    def __init__(self, td):
        self.td = td

    # use the negative log of the tail probability as the score
    def anomaly(self, x):
        cdf = self.td.cdf(x)
        # Here we'll assume we're only testing for "large positive" anomalies
        t = 1 - cdf
        # make sure we don't try to take the logarithm of zero
        t = max(t, 1e-100)
        return -math.log(t)

# Create a Detector
In this cell we'll create an anomaly detector from our data sketch.
Based on the data range we observed above, we declare some testing data points to examine the behavior of our anomaly score.

In [ ]:
detector = TDigestAnomalyDetector(sketch)
data = [0, 5, 10, 15, 20, 25]

# Anomaly Score
The following table shows the numeric behavior of our t-digest based anomaly scores.
As points move out onto the tail of the distribution, the score grows larger, as with our Gaussian anomalies.

The last point has numerically saturated at cdf(x) = 1
(230.2 is the negative log of our safety limit 1e-100).
This is one noteworthy property of using distribution sketches with finite support,
as contrasted with infinite-support models such as a Gaussian.

In [ ]:
scores = pd.DataFrame({
  'x': data,
  'score': [detector.anomaly(x) for x in data]
})
scores

# Plotting the Anomaly Score
Plotting the scores we obtained above gives us a visual intuition about negative-log scores with a t-digest.
Until the score saturates at 1, its behavior is essentially the same as what we saw with Gaussian CDFs.
Because t-digests sketch over a finite support interval, CDF scores will always eventually saturate at 1,
which can result in the large jump we see at our final test point (clipped in the plot below).

In [ ]:
alt.Chart(scores).mark_line(point=True,clip=True).encode(
    alt.Y('score', scale=alt.Scale(domain=(0, 10))),
    x='x'
)

# Exercises
1. If you run this notebook using a larger sample for creating `sketch`, what happens to the support of the distribution?
1. How do the sketch and anomaly score change if the `compression` parameter of the t-digest is altered?
1. Try repeating these experiments with a real data set. Does the behavior of the anomaly score change with different data?